# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [89]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [90]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Pressure,Country,Date
0,0,Yellowknife,62.4560,-114.3525,-12.48,77,75,5.75,1032,CA,1734400602
1,1,Ribeira Grande,38.5167,-28.7000,63.77,76,8,15.01,1012,PT,1734400645
2,2,Tobelo,1.7284,128.0095,85.55,70,95,2.98,1008,ID,1734400448
3,3,Turuntayevo,52.2000,107.6167,11.21,98,100,1.57,1046,RU,1734400648
4,4,Anadyr,64.7500,177.4833,19.90,73,100,17.90,1012,RU,1734400649
5,5,Grand Falls-Windsor,48.9332,-55.6649,30.51,74,100,1.01,1026,CA,1734400650
6,6,Coquimbo,-29.9533,-71.3436,58.73,81,0,4.61,1014,CL,1734400532
7,7,Adamstown,-25.0660,-130.1015,72.07,70,8,14.88,1013,PN,1734400374
8,8,Punta Arenas,-53.1500,-70.9167,53.71,76,0,8.05,1008,CL,1734400653
9,9,Chekmagush,55.1319,54.6556,24.64,90,90,13.47,999,RU,1734400654


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [91]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 0.8,
    color = "City",
    alpha = 0.8,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [98]:
# Narrow down cities that fit criteria and drop any results with null values
#A max temperature lower than 27 degrees but higher than 21,Wind speed less than 4.5 m/s, zero cloudness
idealweather_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 40) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]
#using below 50 and above 0 degree temperature because I had only 2 cities for above 21 and below 27 degree temperature

# Drop any rows with null values
idealweather_city_df = idealweather_city_df.dropna()

# Display sample data
idealweather_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Pressure,Country,Date
13,13,Egilsstadir,65.2653,-14.3948,22.91,86,0,3.44,1003,IS,1734400659
44,44,Turpan,42.9333,89.1667,21.47,58,0,1.52,1041,CN,1734400695
305,305,Szigetvár,46.0486,17.8055,39.61,69,0,2.19,1029,HU,1734400997
449,449,Dêqên,29.9618,90.7188,31.48,15,0,2.15,1019,CN,1734401165


### Step 3: Create a new DataFrame called `hotel_df`.

In [99]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idealweather_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
13,Egilsstadir,IS,65.2653,-14.3948,86,
44,Turpan,CN,42.9333,89.1667,58,
305,Szigetvár,HU,46.0486,17.8055,69,
449,Dêqên,CN,29.9618,90.7188,15,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [100]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Egilsstadir - nearest hotel: Valaskjálf
Turpan - nearest hotel: Hua Sheng Hotel
Szigetvár - nearest hotel: Kumilla Hotel
Dêqên - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
13,Egilsstadir,IS,65.2653,-14.3948,86,Valaskjálf
44,Turpan,CN,42.9333,89.1667,58,Hua Sheng Hotel
305,Szigetvár,HU,46.0486,17.8055,69,Kumilla Hotel
449,Dêqên,CN,29.9618,90.7188,15,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [101]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)